<a href="https://colab.research.google.com/github/larry-tableau/tableau/blob/main/Banking_Sample_Data_Set_into_BQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pantab==4.1.0
!pip --quiet install pantab
!pip --quiet install tableauhyperapi
!pip --quiet install pandas faker
!pip --quiet install --upgrade Faker
!pip --quiet install Mimesis
!pip --quiet install --upgrade numpy
import requests
import json
import pandas as pd
import pantab
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 107.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, date
import string
import warnings
import os
import json

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Configuration parameters - easily adjustable
NUM_ROWS = 150000
START_DATE = datetime(2022, 6, 7)  # 3 years before reference date
END_DATE = datetime(2025, 6, 7)    # Reference date
OUTLIER_FRACTION = 0.02  # 2% of records will have outliers
OUTLIER_MAGNITUDE = 5.0  # Outliers will be 5x normal values

# BigQuery configuration
PROJECT_ID = "ehc-ldioneda-1d3210"
DATASET_ID = "larry_demo_datasets"
TABLE_ID = "bb_banking_dataset_sample"
FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

def setup_colab_authentication():
    """
    Setup authentication in Google Colab environment
    """
    print("Setting up Google Colab authentication...")

    try:
        # Method 1: Try user authentication first
        from google.colab import auth
        print("🔑 Attempting user authentication...")
        auth.authenticate_user()
        print("✅ User authentication successful!")
        return "user"
    except Exception as e:
        print(f"⚠️  User authentication failed: {e}")

    # Method 2: Service account upload
    print("\n📁 Please upload your service account JSON file using the method below:")
    print("1. Click the 'Choose Files' button that appears")
    print("2. Select your service account JSON file")
    print("3. Wait for upload to complete")

    try:
        from google.colab import files
        uploaded = files.upload()

        if uploaded:
            # Get the first uploaded file
            filename = list(uploaded.keys())[0]
            print(f"📄 Uploaded file: {filename}")

            # Set environment variable
            os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = filename
            print("✅ Service account credentials configured!")
            return "service_account"
        else:
            print("❌ No file uploaded")
            return None

    except Exception as e:
        print(f"❌ File upload failed: {e}")
        return None

def setup_bigquery_client():
    """
    Setup BigQuery client for Google Colab
    """
    try:
        from google.cloud import bigquery
        from google.cloud.exceptions import NotFound
        from google.auth.exceptions import DefaultCredentialsError

        print(f"Setting up BigQuery client for project: {PROJECT_ID}")

        # Try to create client
        client = bigquery.Client(project=PROJECT_ID)

        # Test the connection
        try:
            dataset = client.get_dataset(DATASET_ID)
            print(f"✅ Successfully connected to dataset: {DATASET_ID}")
            return client, bigquery, NotFound
        except Exception as e:
            print(f"⚠️  Dataset access test failed: {e}")
            print("Note: This might be normal if the dataset exists but has restricted access")
            return client, bigquery, NotFound

    except ImportError:
        print("❌ google-cloud-bigquery not installed. Installing now...")
        !pip install google-cloud-bigquery

        from google.cloud import bigquery
        from google.cloud.exceptions import NotFound
        client = bigquery.Client(project=PROJECT_ID)
        return client, bigquery, NotFound

    except Exception as e:
        print(f"❌ Error setting up BigQuery client: {e}")
        print("\nTrying alternative authentication...")
        return None, None, None

def create_bigquery_table_schema():
    """
    Define BigQuery table schema based on the generated dataset
    """
    # Import here in case it's not available at module level
    from google.cloud import bigquery

    schema = [
        bigquery.SchemaField("evaluate_date", "DATE", mode="REQUIRED"),
        bigquery.SchemaField("customer_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("relation_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("base_customer", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("date_added", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("delete_flag", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("cust_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("cust_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("address", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("city", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("state", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("postcode", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("branch", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("operating_branch", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("salutation", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("class_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("sic_code", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("customer_category", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("charge_officer1", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("account_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("institution", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("folio", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("ledger", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("account", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("bank_code", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("account_status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("product", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("prod_lvl1", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("prod_lvl2", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("prod_lvl3", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("prod_lvl4", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("prod_grp", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("bgod", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("custacc", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("oic_report", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("business_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("footing_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("balance", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("footing", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("current_limit", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("maturity_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("tran_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("tran_method", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("transaction_description", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("actual_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("tran_eff_date", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("tran_amount", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("amount", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("dr_cr", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("terminal_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("card_bin_no", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("reference_number", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("reference_data", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("reference", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("oic1_new", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("name_new", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("oic_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("oic1", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("adm_new", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("adm", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_util_new", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_cost_code", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_utility_code", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("effective_from", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("effective_to", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml1_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml1_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml2_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml2_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml3_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml3_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml4_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml4_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml5_id", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gl_ml5_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("rel_mgt_class", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("direct_report_adm", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("direct_report_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("direct_report_title", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("direct_report_adm_overwrite", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("curr_flag", "BOOLEAN", mode="NULLABLE"),
        bigquery.SchemaField("oic_and_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("event_timestamp_utc", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("avg_products_per_customer", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("proactive_opportunity_win_rate", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("key_product_portfolio_growth", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("competitor_win_back_rate", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("opportunity_source", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("displaced_competitor", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("strategic_product_group", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("time_granularity", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("reporting_period_start", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("reporting_period_end", "DATE", mode="NULLABLE"),
    ]
    return schema

def create_or_update_bigquery_table(client, table_id, schema, bigquery, NotFound):
    """
    Create BigQuery table if it doesn't exist, or update schema if it does
    """
    try:
        # Check if table exists
        table = client.get_table(table_id)
        print(f"Table {table_id} already exists. Checking schema...")

        # Check if schema needs updating
        current_schema = table.schema
        if len(current_schema) != len(schema):
            print("Schema differs. Updating table schema...")
            table.schema = schema
            table = client.update_table(table, ["schema"])
            print("Table schema updated successfully.")
        else:
            print("Table schema is up to date.")

    except NotFound:
        # Table doesn't exist, create it
        print(f"Creating table {table_id}...")
        table = bigquery.Table(table_id, schema=schema)
        table = client.create_table(table)
        print(f"Table {table_id} created successfully.")

    return table

def upload_to_bigquery(df, client, table_id, bigquery, write_disposition="WRITE_TRUNCATE"):
    """
    Upload DataFrame to BigQuery table
    """
    # Configure the job
    job_config = bigquery.LoadJobConfig(
        write_disposition=write_disposition,
        autodetect=False,  # Use our explicit schema
        schema=create_bigquery_table_schema()
    )

    print(f"Starting upload to {table_id}...")
    print(f"Upload mode: {write_disposition}")
    print(f"Rows to upload: {len(df):,}")

    # Start the job
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

    # Wait for the job to complete
    try:
        job.result()  # Wait for job to complete
        print(f"✅ Successfully uploaded {len(df):,} rows to {table_id}")

        # Get table info
        table = client.get_table(table_id)
        print(f"📊 Table now contains {table.num_rows:,} total rows")
        return True

    except Exception as e:
        print(f"❌ Error uploading to BigQuery: {e}")
        if hasattr(job, 'errors') and job.errors:
            for error in job.errors:
                print(f"Job error: {error}")
        return False

def generate_bendigo_adelaide_banking_data():
    """
    Generate comprehensive banking dataset for Bendigo and Adelaide Bank
    Returns: pandas DataFrame with 150,000 rows and 87 columns
    """

    print(f"Generating {NUM_ROWS:,} rows of Bendigo and Adelaide Bank portfolio data...")

    # Australian states and territories
    australian_states = ['VIC', 'NSW', 'QLD', 'WA', 'SA', 'TAS', 'NT', 'ACT']

    # Australian cities weighted by population
    australian_cities = [
        'Melbourne', 'Sydney', 'Brisbane', 'Perth', 'Adelaide', 'Gold Coast',
        'Newcastle', 'Canberra', 'Sunshine Coast', 'Wollongong', 'Hobart',
        'Geelong', 'Townsville', 'Cairns', 'Darwin', 'Toowoomba', 'Ballarat',
        'Bendigo', 'Albury', 'Launceston', 'Mackay', 'Rockhampton', 'Bunbury',
        'Bundaberg', 'Coffs Harbour', 'Wagga Wagga', 'Hervey Bay', 'Mildura',
        'Shepparton', 'Port Macquarie', 'Gladstone', 'Tamworth', 'Traralgon',
        'Orange', 'Bowral', 'Geraldton', 'Dubbo', 'Nowra', 'Warrnambool'
    ]

    # Bendigo and Adelaide Bank branch names (realistic Australian locations)
    branch_names = [
        'Melbourne CBD', 'Sydney CBD', 'Brisbane CBD', 'Perth CBD', 'Adelaide CBD',
        'Bendigo Central', 'Ballarat', 'Geelong', 'Shepparton', 'Mildura',
        'Echuca', 'Swan Hill', 'Horsham', 'Warrnambool', 'Hamilton',
        'Castlemaine', 'Kyneton', 'Woodend', 'Gisborne', 'Sunbury',
        'Melton', 'Bacchus Marsh', 'Ararat', 'Stawell', 'Nhill',
        'Adelaide Hills', 'Mount Gambier', 'Port Augusta', 'Whyalla', 'Port Pirie',
        'Murray Bridge', 'Victor Harbor', 'Gawler', 'Mount Barker', 'Berri',
        'Renmark', 'Loxton', 'Naracoorte', 'Millicent', 'Bordertown',
        'Kadina', 'Clare', 'Jamestown', 'Peterborough', 'Leigh Creek',
        'Coober Pedy', 'Roxby Downs', 'Wudinna', 'Streaky Bay', 'Ceduna',
        'Broken Hill', 'Mildura West', 'Swan Hill East', 'Echuca South',
        'Bendigo North', 'Bendigo South', 'Bendigo East', 'Bendigo West',
        'Kangaroo Flat', 'Eaglehawk', 'Strathfieldsaye', 'Golden Square',
        'White Hills', 'Quarry Hill', 'Flora Hill', 'Kennington',
        'Huntly', 'Marong', 'Heathcote', 'Elmore', 'Rochester', 'Lockington'
    ]

    # Relationship Manager names (realistic Australian names)
    rm_first_names = [
        'Andrew', 'Sarah', 'Michael', 'Emma', 'David', 'Jessica', 'James', 'Rebecca',
        'Matthew', 'Lisa', 'Daniel', 'Michelle', 'Christopher', 'Amanda', 'Anthony',
        'Nicole', 'Mark', 'Samantha', 'Paul', 'Rachel', 'Steven', 'Catherine',
        'Peter', 'Jennifer', 'Robert', 'Melissa', 'John', 'Stephanie', 'Gary',
        'Natalie', 'Simon', 'Vanessa', 'Luke', 'Tanya', 'Adam', 'Joanne',
        'Benjamin', 'Kylie', 'Nathan', 'Tracey', 'Ryan', 'Leanne', 'Scott',
        'Donna', 'Timothy', 'Sharon', 'Jason', 'Karen', 'Brett', 'Susan'
    ]

    rm_last_names = [
        'Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller',
        'Davis', 'Rodriguez', 'Martinez', 'Hernandez', 'Lopez', 'Gonzalez',
        'Wilson', 'Anderson', 'Thomas', 'Taylor', 'Moore', 'Jackson', 'Martin',
        'Lee', 'Perez', 'Thompson', 'White', 'Harris', 'Sanchez', 'Clark',
        'Ramirez', 'Lewis', 'Robinson', 'Walker', 'Young', 'Allen', 'King',
        'Wright', 'Scott', 'Torres', 'Nguyen', 'Hill', 'Flores', 'Green',
        'Adams', 'Nelson', 'Baker', 'Hall', 'Rivera', 'Campbell', 'Mitchell',
        'Carter', 'Roberts', 'Gomez', 'Phillips', 'Evans', 'Turner', 'Diaz'
    ]

    # Product hierarchy data
    product_hierarchy = {
        'Loans': {
            'Home Loans': {
                'Variable Rate': ['Standard Variable', 'Premium Variable', 'Introductory Variable'],
                'Fixed Rate': ['1 Year Fixed', '2 Year Fixed', '3 Year Fixed', '5 Year Fixed']
            },
            'Business Loans': {
                'Term Loans': ['Equipment Finance', 'Commercial Property', 'Working Capital'],
                'Line of Credit': ['Business LOC', 'Commercial LOC', 'Agribusiness LOC']
            },
            'Personal Loans': {
                'Secured': ['Car Loan', 'Personal Secured'],
                'Unsecured': ['Personal Unsecured', 'Debt Consolidation']
            }
        },
        'Deposits': {
            'Transaction Accounts': {
                'Everyday Banking': ['Complete Access', 'Bendigo Access', 'Student Access'],
                'Business Banking': ['Business Cheque', 'Business Access', 'Not-for-Profit']
            },
            'Savings': {
                'Term Deposits': ['3 Month TD', '6 Month TD', '12 Month TD', '24 Month TD'],
                'Online Savings': ['eSaver', 'Goal Saver', 'Bonus Saver']
            }
        },
        'Cards': {
            'Credit Cards': {
                'Rewards': ['Platinum Rewards', 'Gold Rewards', 'Classic Rewards'],
                'Low Rate': ['Low Rate Platinum', 'Low Rate Gold']
            },
            'Debit Cards': {
                'EFTPOS': ['Bendigo Debit', 'Business Debit'],
                'Visa Debit': ['Visa Debit', 'Business Visa Debit']
            }
        }
    }

    # Generate base data structure
    data = {}

    # Date fields - evaluate_date with clustering at month-ends
    print("Generating date fields...")
    evaluate_dates = []
    for _ in range(NUM_ROWS):
        if random.random() < 0.3:  # 30% chance of month-end clustering
            # Generate month-end dates
            year = random.randint(2022, 2025)
            month = random.randint(1, 12)
            if month == 12:
                last_day = 31
            elif month in [4, 6, 9, 11]:
                last_day = 30
            elif month == 2:
                last_day = 28 if year % 4 != 0 else 29
            else:
                last_day = 31
            day = random.randint(max(1, last_day-2), last_day)
            evaluate_dates.append(date(year, month, day))
        else:
            # Random dates within range
            days_diff = (END_DATE.date() - START_DATE.date()).days
            random_days = random.randint(0, days_diff)
            evaluate_dates.append(START_DATE.date() + timedelta(days=random_days))

    data['evaluate_date'] = evaluate_dates

    # Customer identification fields
    print("Generating customer identification...")
    data['customer_id'] = [f"CUST{str(i).zfill(6)}" for i in range(1, NUM_ROWS + 1)]
    data['relation_type'] = np.random.choice(['Primary', 'Joint'], NUM_ROWS, p=[0.8, 0.2])
    data['base_customer'] = np.random.choice(['Y', 'N'], NUM_ROWS, p=[0.9, 0.1])

    # Date added - uniform distribution over past 5 years
    date_added_start = datetime(2020, 6, 5)
    date_added_list = []
    for _ in range(NUM_ROWS):
        days_diff = (END_DATE - date_added_start).days
        random_days = random.randint(0, days_diff)
        date_added_list.append((date_added_start + timedelta(days=random_days)).date())
    data['date_added'] = date_added_list

    data['delete_flag'] = np.random.choice([False, True], NUM_ROWS, p=[0.98, 0.02])
    data['cust_type'] = np.random.choice(['Individual', 'Business'], NUM_ROWS, p=[0.7, 0.3])

    # Customer names based on type
    print("Generating customer names...")
    customer_names = []
    for cust_type in data['cust_type']:
        if cust_type == 'Individual':
            first_name = random.choice(rm_first_names)
            last_name = random.choice(rm_last_names)
            customer_names.append(f"{first_name} {last_name}")
        else:
            business_types = ['Pty Ltd', 'Pty Limited', 'Limited', 'P/L', 'Trust', 'Partnership']
            business_names = [
                'Sunrise', 'Golden', 'Premier', 'Elite', 'Advanced', 'Professional',
                'Quality', 'Superior', 'Excellence', 'Innovation', 'Dynamic', 'Strategic',
                'Progressive', 'Modern', 'Classic', 'Heritage', 'Future', 'Vision'
            ]
            business_sectors = [
                'Consulting', 'Services', 'Solutions', 'Industries', 'Enterprises',
                'Holdings', 'Group', 'Partners', 'Associates', 'Ventures', 'Capital',
                'Investments', 'Development', 'Construction', 'Manufacturing', 'Trading'
            ]
            name = f"{random.choice(business_names)} {random.choice(business_sectors)} {random.choice(business_types)}"
            customer_names.append(name)
    data['cust_name'] = customer_names

    # Address information
    print("Generating address information...")
    street_numbers = [str(random.randint(1, 999)) for _ in range(NUM_ROWS)]
    street_names = [
        'Collins', 'Bourke', 'Flinders', 'Elizabeth', 'Queen', 'King', 'William',
        'Spencer', 'Russell', 'Swanston', 'Exhibition', 'Spring', 'Lonsdale',
        'Little Collins', 'Little Bourke', 'La Trobe', 'Franklin', 'A\'Beckett',
        'Flagstaff', 'Dudley', 'Hardware', 'Rundle', 'Hindley', 'Currie',
        'Grenfell', 'Pirie', 'Waymouth', 'Gouger', 'Grote', 'Angas'
    ]
    street_types = ['Street', 'Road', 'Avenue', 'Drive', 'Lane', 'Court', 'Place', 'Way']

    addresses = []
    for i in range(NUM_ROWS):
        street_num = street_numbers[i]
        street_name = random.choice(street_names)
        street_type = random.choice(street_types)
        addresses.append(f"{street_num} {street_name} {street_type}")
    data['address'] = addresses

    # Cities weighted by population - FIXED: 39 cities with 39 weights (5 + 34)
    city_weights = [0.25, 0.20, 0.15, 0.10, 0.08] + [0.22/34] * 34  # Top 5 cities get higher weights
    data['city'] = np.random.choice(australian_cities, NUM_ROWS, p=city_weights)

    # States based on cities (simplified mapping)
    state_mapping = {
        'Melbourne': 'VIC', 'Geelong': 'VIC', 'Ballarat': 'VIC', 'Bendigo': 'VIC',
        'Sydney': 'NSW', 'Newcastle': 'NSW', 'Wollongong': 'NSW', 'Albury': 'NSW',
        'Brisbane': 'QLD', 'Gold Coast': 'QLD', 'Sunshine Coast': 'QLD', 'Townsville': 'QLD',
        'Perth': 'WA', 'Bunbury': 'WA', 'Geraldton': 'WA',
        'Adelaide': 'SA', 'Mount Gambier': 'SA',
        'Hobart': 'TAS', 'Launceston': 'TAS',
        'Darwin': 'NT',
        'Canberra': 'ACT'
    }

    states = []
    for city in data['city']:
        if city in state_mapping:
            states.append(state_mapping[city])
        else:
            states.append(random.choice(australian_states))
    data['state'] = states

    # Australian postcodes (simplified - using realistic ranges)
    postcode_ranges = {
        'VIC': (3000, 3999), 'NSW': (2000, 2999), 'QLD': (4000, 4999),
        'WA': (6000, 6999), 'SA': (5000, 5999), 'TAS': (7000, 7999),
        'NT': (800, 999), 'ACT': (2600, 2699)
    }

    postcodes = []
    for state in data['state']:
        min_pc, max_pc = postcode_ranges[state]
        postcodes.append(str(random.randint(min_pc, max_pc)))
    data['postcode'] = postcodes

    # Branch assignment
    data['branch'] = np.random.choice(branch_names, NUM_ROWS)
    data['operating_branch'] = [branch if random.random() < 0.9 else random.choice(branch_names)
                               for branch in data['branch']]

    # Salutations
    salutations = ['Mr', 'Ms', 'Mrs', 'Dr', 'Prof', 'Miss']
    data['salutation'] = np.random.choice(salutations, NUM_ROWS)

    # Customer classification
    data['class_type'] = np.random.choice(['Platinum', 'Gold', 'Standard'], NUM_ROWS, p=[0.1, 0.2, 0.7])

    # SIC codes for business customers only
    sic_codes = ['0111', '0112', '0115', '1311', '1389', '2211', '2212', '2213', '3311', '4111']
    sic_code_list = []
    for cust_type in data['cust_type']:
        if cust_type == 'Business':
            sic_code_list.append(random.choice(sic_codes))
        else:
            sic_code_list.append(None)
    data['sic_code'] = sic_code_list

    # Customer category
    data['customer_category'] = np.random.choice(['Retail', 'Business', 'Agribusiness'], NUM_ROWS, p=[0.6, 0.3, 0.1])

    # Relationship Managers
    print("Generating relationship manager data...")
    rm_names = [f"{random.choice(rm_first_names)} {random.choice(rm_last_names)}" for _ in range(150)]
    data['charge_officer1'] = np.random.choice(rm_names, NUM_ROWS)

    # Account information
    print("Generating account information...")
    data['account_id'] = [f"ACC{str(i).zfill(6)}" for i in range(1, NUM_ROWS + 1)]
    data['institution'] = np.random.choice(['BENDIGO', 'ADELAIDE', 'RURAL'], NUM_ROWS, p=[0.6, 0.3, 0.1])
    data['folio'] = [str(random.randint(100000, 999999)) for _ in range(NUM_ROWS)]
    data['ledger'] = [''.join(random.choices(string.ascii_uppercase, k=3)) for _ in range(NUM_ROWS)]
    data['account'] = [str(random.randint(10000000, 999999999)) for _ in range(NUM_ROWS)]

    # Bank codes
    bank_code_mapping = {'BENDIGO': 'BEND', 'ADELAIDE': 'ADLB', 'RURAL': 'RUR'}
    data['bank_code'] = [bank_code_mapping[inst] for inst in data['institution']]

    # Account status
    data['account_status'] = np.random.choice(['Active', 'Closed', 'Dormant'], NUM_ROWS, p=[0.85, 0.10, 0.05])

    # Product hierarchy generation
    print("Generating product hierarchy...")
    products = []
    prod_lvl1s = []
    prod_lvl2s = []
    prod_lvl3s = []
    prod_lvl4s = []
    prod_grps = []

    for _ in range(NUM_ROWS):
        lvl1 = random.choice(list(product_hierarchy.keys()))
        lvl2 = random.choice(list(product_hierarchy[lvl1].keys()))
        lvl3 = random.choice(list(product_hierarchy[lvl1][lvl2].keys()))
        lvl4 = random.choice(product_hierarchy[lvl1][lvl2][lvl3])

        products.append(f"{lvl1} - {lvl2} - {lvl3} - {lvl4}")
        prod_lvl1s.append(lvl1)
        prod_lvl2s.append(lvl2)
        prod_lvl3s.append(lvl3)
        prod_lvl4s.append(lvl4)
        prod_grps.append(f"{lvl1[:2]}{lvl2[:2]}".upper())

    data['product'] = products
    data['prod_lvl1'] = prod_lvl1s
    data['prod_lvl2'] = prod_lvl2s
    data['prod_lvl3'] = prod_lvl3s
    data['prod_lvl4'] = prod_lvl4s
    data['prod_grp'] = prod_grps
    data['type'] = np.random.choice(['Retail', 'Business'], NUM_ROWS, p=[0.7, 0.3])

    # Internal codes
    data['bgod'] = [''.join(random.choices(string.ascii_uppercase, k=4)) for _ in range(NUM_ROWS)]
    data['custacc'] = [f"CA{random.randint(10000, 99999)}" for _ in range(NUM_ROWS)]
    data['oic_report'] = [f"OIC{random.randint(100, 999)}" for _ in range(NUM_ROWS)]

    # Business dates
    business_dates = []
    for eval_date in data['evaluate_date']:
        # Business date should be close to evaluate date
        days_offset = random.randint(-5, 5)
        business_dates.append(eval_date + timedelta(days=days_offset))
    data['business_date'] = business_dates

    # Footing types
    data['footing_type'] = np.random.choice(['Book', 'Available'], NUM_ROWS, p=[0.7, 0.3])

    # Financial measures
    print("Generating financial measures...")

    # Balance - log-normal distribution with outliers
    base_balances = np.random.lognormal(mean=8.5, sigma=1.5, size=NUM_ROWS)

    # Add outliers for recent dates (last 30 days)
    recent_date_threshold = END_DATE.date() - timedelta(days=30)
    outlier_indices = []

    for i, eval_date in enumerate(data['evaluate_date']):
        if eval_date >= recent_date_threshold and random.random() < OUTLIER_FRACTION:
            outlier_indices.append(i)

    balances = base_balances.copy()
    for idx in outlier_indices:
        balances[idx] *= OUTLIER_MAGNITUDE

    data['balance'] = np.round(balances, 2)

    # Footing - correlated with balance but can differ
    footings = []
    for balance in data['balance']:
        correlation_factor = random.uniform(0.8, 1.2)
        footings.append(round(balance * correlation_factor, 2))
    data['footing'] = footings

    # Current limit - 0 for deposits, positive for loans
    current_limits = []
    for prod_lvl1 in data['prod_lvl1']:
        if prod_lvl1 == 'Loans':
            current_limits.append(round(random.uniform(10000, 1000000), 2))
        else:
            current_limits.append(0.0)
    data['current_limit'] = current_limits

    # Maturity dates for term products
    maturity_dates = []
    for i, (date_added, prod_lvl3) in enumerate(zip(data['date_added'], data['prod_lvl3'])):
        if 'Term' in prod_lvl3 or 'Fixed' in prod_lvl3:
            years_to_add = random.randint(1, 5)
            maturity_dates.append(date_added + timedelta(days=years_to_add*365))
        else:
            maturity_dates.append(None)
    data['maturity_date'] = maturity_dates

    # Transaction information
    print("Generating transaction data...")
    data['tran_type'] = np.random.choice(['Deposit', 'Withdrawal', 'Payment'], NUM_ROWS, p=[0.6, 0.3, 0.1])
    data['tran_method'] = np.random.choice(['Online', 'Branch', 'ATM'], NUM_ROWS, p=[0.5, 0.3, 0.2])

    # Transaction descriptions
    transaction_descriptions = [
        'Direct Debit Payment', 'EFTPOS Purchase', 'ATM Withdrawal', 'Online Transfer',
        'Salary Credit', 'Interest Payment', 'Fee Debit', 'Cheque Deposit',
        'Card Payment', 'Bill Payment', 'Loan Repayment', 'Term Deposit Maturity'
    ]
    data['transaction_description'] = np.random.choice(transaction_descriptions, NUM_ROWS)

    # Transaction dates
    actual_dates = []
    tran_eff_dates = []
    for business_date in data['business_date']:
        actual_date = business_date + timedelta(days=random.randint(-1, 1))
        eff_date = actual_date + timedelta(days=random.randint(0, 2))
        actual_dates.append(actual_date)
        tran_eff_dates.append(eff_date)

    data['actual_date'] = actual_dates
    data['tran_eff_date'] = tran_eff_dates

    # Transaction amounts - skewed distribution with outliers
    base_tran_amounts = np.random.gamma(2, 500, NUM_ROWS)

    # Add outliers for recent transactions
    tran_amounts = base_tran_amounts.copy()
    for idx in outlier_indices:
        if random.random() < 0.5:  # 50% chance for transaction outliers
            tran_amounts[idx] *= OUTLIER_MAGNITUDE

    data['tran_amount'] = np.round(tran_amounts, 2)
    data['amount'] = data['tran_amount']  # Duplicate field

    # Debit/Credit indicator
    data['dr_cr'] = np.random.choice(['DR', 'CR'], NUM_ROWS, p=[0.5, 0.5])

    # Terminal and card information
    data['terminal_type'] = np.random.choice(['ATM', 'POS', 'Online'], NUM_ROWS, p=[0.3, 0.4, 0.3])
    data['card_bin_no'] = [str(random.randint(400000, 599999)) for _ in range(NUM_ROWS)]

    # Reference fields
    data['reference_number'] = [f"REF{random.randint(1000000, 9999999)}" for _ in range(NUM_ROWS)]
    data['reference_data'] = [f"RD{random.randint(10000, 99999)}" for _ in range(NUM_ROWS)]
    data['reference'] = [f"R{random.randint(1000, 9999)}" for _ in range(NUM_ROWS)]

    # Officer information
    print("Generating officer and management data...")

    # Generate unique RM codes and names
    unique_rms = list(set(data['charge_officer1']))
    rm_codes = {name: f"RM{str(i).zfill(3)}" for i, name in enumerate(unique_rms)}

    data['oic1_new'] = [rm_codes[rm] for rm in data['charge_officer1']]
    data['name_new'] = data['charge_officer1']
    data['oic_name'] = data['charge_officer1']  # Legacy field
    data['oic1'] = data['oic1_new']  # Legacy field
    data['name'] = data['charge_officer1']  # Legacy field

    # Administrative codes
    data['adm_new'] = [f"ADM{random.randint(100, 999)}" for _ in range(NUM_ROWS)]
    data['adm'] = data['adm_new']  # Legacy field
    data['gl_util_new'] = [f"GL{random.randint(1000, 9999)}" for _ in range(NUM_ROWS)]

    # Titles
    titles = ['Relationship Manager', 'Senior Relationship Manager', 'Team Leader',
              'Regional Manager', 'Area Manager', 'Business Banking Manager']
    data['title'] = np.random.choice(titles, NUM_ROWS)

    # GL codes
    data['gl_cost_code'] = [f"CC{random.randint(1000, 9999)}" for _ in range(NUM_ROWS)]
    data['gl_utility_code'] = [f"GU{random.randint(1000, 9999)}" for _ in range(NUM_ROWS)]

    # Effective dates for RM assignments
    effective_from_dates = []
    effective_to_dates = []

    for _ in range(NUM_ROWS):
        from_date = START_DATE.date() + timedelta(days=random.randint(0, 1000))
        effective_from_dates.append(from_date)

        # 80% current assignments (NULL to date), 20% past assignments
        if random.random() < 0.8:
            effective_to_dates.append(None)
        else:
            to_date = from_date + timedelta(days=random.randint(30, 730))
            effective_to_dates.append(to_date)

    data['effective_from'] = effective_from_dates
    data['effective_to'] = effective_to_dates

    # GL multi-level hierarchy
    gl_ml1_options = ['Retail Banking', 'Business Banking', 'Agribusiness', 'Wealth Management']
    gl_ml2_options = ['Metro', 'Regional', 'Rural', 'Corporate']
    gl_ml3_options = ['Victoria', 'NSW', 'Queensland', 'South Australia', 'Western Australia']
    gl_ml4_options = ['Melbourne', 'Sydney', 'Brisbane', 'Adelaide', 'Perth', 'Bendigo']
    gl_ml5_options = ['CBD', 'Suburban', 'Rural', 'Industrial']

    data['gl_ml1_id'] = [f"ML1{random.randint(100, 999)}" for _ in range(NUM_ROWS)]
    data['gl_ml1_name'] = np.random.choice(gl_ml1_options, NUM_ROWS)
    data['gl_ml2_id'] = [f"ML2{random.randint(100, 999)}" for _ in range(NUM_ROWS)]
    data['gl_ml2_name'] = np.random.choice(gl_ml2_options, NUM_ROWS)
    data['gl_ml3_id'] = [f"ML3{random.randint(100, 999)}" for _ in range(NUM_ROWS)]
    data['gl_ml3_name'] = np.random.choice(gl_ml3_options, NUM_ROWS)
    data['gl_ml4_id'] = [f"ML4{random.randint(100, 999)}" for _ in range(NUM_ROWS)]
    data['gl_ml4_name'] = np.random.choice(gl_ml4_options, NUM_ROWS)
    data['gl_ml5_id'] = [f"ML5{random.randint(100, 999)}" for _ in range(NUM_ROWS)]
    data['gl_ml5_name'] = np.random.choice(gl_ml5_options, NUM_ROWS)

    # Management hierarchy
    data['rel_mgt_class'] = np.random.choice(['Senior RM', 'Junior RM', 'Graduate RM'], NUM_ROWS, p=[0.4, 0.5, 0.1])
    data['direct_report_adm'] = [f"DRA{random.randint(100, 999)}" for _ in range(NUM_ROWS)]
    data['direct_report_name'] = np.random.choice(rm_names[:50], NUM_ROWS)  # Subset for managers
    data['direct_report_title'] = np.random.choice(['Regional Manager', 'Area Manager', 'General Manager'], NUM_ROWS)
    data['direct_report_adm_overwrite'] = [f"DRO{random.randint(100, 999)}" for _ in range(NUM_ROWS)]

    # Current flags
    data['curr_flag'] = np.random.choice([True, False], NUM_ROWS, p=[0.95, 0.05])

    # Combined OIC and name field
    data['oic_and_name'] = [f"{oic} - {name}" for oic, name in zip(data['oic1_new'], data['name_new'])]

    # Event timestamps
    event_timestamps = []
    for business_date in data['business_date']:
        # Random time within business hours (9 AM to 5 PM)
        hour = random.randint(9, 17)
        minute = random.randint(0, 59)
        second = random.randint(0, 59)
        timestamp = datetime.combine(business_date, datetime.min.time().replace(hour=hour, minute=minute, second=second))
        event_timestamps.append(timestamp)

    data['event_timestamp_utc'] = event_timestamps

    # KPI measures
    print("Generating KPI measures...")

    # Average products per customer - normal distribution
    avg_products = np.random.normal(2.5, 1.0, NUM_ROWS)
    avg_products = np.clip(avg_products, 1.0, 6.0)  # Cap at 6
    data['avg_products_per_customer'] = np.round(avg_products, 2)

    # Proactive opportunity win rate - beta distribution skewed high
    win_rates = np.random.beta(3, 2, NUM_ROWS) * 50 + 40  # 40-90% range
    data['proactive_opportunity_win_rate'] = np.round(win_rates, 2)

    # Key product portfolio growth - positive skewed integers
    portfolio_growth = np.random.gamma(2, 5, NUM_ROWS)
    portfolio_growth = np.clip(portfolio_growth, 0, 50)
    data['key_product_portfolio_growth'] = np.round(portfolio_growth, 0)

    # Competitor win back rate
    win_back_rates = np.random.uniform(0, 30, NUM_ROWS)
    # Add spikes for campaign periods (recent dates)
    for i, eval_date in enumerate(data['evaluate_date']):
        if eval_date >= recent_date_threshold and random.random() < 0.1:
            win_back_rates[i] *= 2  # Double the rate during campaigns

    data['competitor_win_back_rate'] = np.round(win_back_rates, 2)

    # Opportunity sources
    opportunity_sources = ['Analytics Alert', 'LinkedIn Signal', 'Customer Inquiry', 'Referral', 'Campaign Response']
    data['opportunity_source'] = np.random.choice(opportunity_sources, NUM_ROWS, p=[0.6, 0.3, 0.05, 0.03, 0.02])

    # Displaced competitors
    competitors = ['Commonwealth Bank', 'ANZ', 'Westpac', 'NAB', 'ING', 'Macquarie Bank', 'Suncorp', 'Bank of Queensland']
    data['displaced_competitor'] = np.random.choice(competitors, NUM_ROWS)

    # Strategic product groups
    strategic_groups = ['Business Loans', 'Wealth Solutions', 'Agribusiness Finance', 'Home Loans', 'Transaction Banking']
    data['strategic_product_group'] = np.random.choice(strategic_groups, NUM_ROWS)

    # Time granularity and reporting periods
    time_granularities = ['Daily', 'Weekly', 'Monthly', 'Quarterly']
    data['time_granularity'] = np.random.choice(time_granularities, NUM_ROWS, p=[0.1, 0.2, 0.5, 0.2])

    # Reporting period start and end dates
    reporting_start_dates = []
    reporting_end_dates = []

    for granularity, business_date in zip(data['time_granularity'], data['business_date']):
        if granularity == 'Daily':
            start_date = business_date
            end_date = business_date
        elif granularity == 'Weekly':
            # Start of week (Monday)
            days_since_monday = business_date.weekday()
            start_date = business_date - timedelta(days=days_since_monday)
            end_date = start_date + timedelta(days=6)
        elif granularity == 'Monthly':
            # Start of month
            start_date = business_date.replace(day=1)
            if start_date.month == 12:
                end_date = start_date.replace(year=start_date.year + 1, month=1) - timedelta(days=1)
            else:
                end_date = start_date.replace(month=start_date.month + 1) - timedelta(days=1)
        else:  # Quarterly
            # Start of quarter
            quarter_start_month = ((business_date.month - 1) // 3) * 3 + 1
            start_date = business_date.replace(month=quarter_start_month, day=1)
            if quarter_start_month == 10:
                end_date = start_date.replace(year=start_date.year + 1, month=1) - timedelta(days=1)
            else:
                end_date = start_date.replace(month=quarter_start_month + 3) - timedelta(days=1)

        reporting_start_dates.append(start_date)
        reporting_end_dates.append(end_date)

    data['reporting_period_start'] = reporting_start_dates
    data['reporting_period_end'] = reporting_end_dates

    # Create DataFrame
    print("Creating DataFrame...")
    df = pd.DataFrame(data)

    # Apply outliers to recent data for key financial measures
    print("Applying outliers to recent data...")
    recent_mask = df['evaluate_date'] >= recent_date_threshold

    # Apply outliers to balance and transaction amounts for recent records
    outlier_mask = recent_mask & (np.random.random(NUM_ROWS) < OUTLIER_FRACTION)

    df.loc[outlier_mask, 'balance'] *= OUTLIER_MAGNITUDE
    df.loc[outlier_mask, 'tran_amount'] *= OUTLIER_MAGNITUDE
    df.loc[outlier_mask, 'amount'] = df.loc[outlier_mask, 'tran_amount']
    df.loc[outlier_mask, 'proactive_opportunity_win_rate'] = np.minimum(
        df.loc[outlier_mask, 'proactive_opportunity_win_rate'] * 1.5, 95
    )

    # Round financial columns
    financial_columns = ['balance', 'footing', 'current_limit', 'tran_amount', 'amount']
    for col in financial_columns:
        df[col] = df[col].round(2)

    print(f"Dataset generation complete! Generated {len(df):,} rows with {len(df.columns)} columns.")
    print(f"Date range: {df['evaluate_date'].min()} to {df['evaluate_date'].max()}")
    print(f"Outliers applied to {outlier_mask.sum():,} recent records ({(outlier_mask.sum()/NUM_ROWS)*100:.1f}%)")

    return df

def save_csv_backup(df, filename='bendigo_adelaide_bank_portfolio_analytics_150k.csv'):
    """
    Save DataFrame as CSV with Australian date formatting
    """
    print(f"\n📁 Saving CSV backup: {filename}")
    df_export = df.copy()

    # Convert date columns to Australian format (DD/MM/YYYY) for CSV export
    date_columns = ['evaluate_date', 'date_added', 'maturity_date', 'business_date',
                   'actual_date', 'tran_eff_date', 'effective_from', 'effective_to',
                   'reporting_period_start', 'reporting_period_end']

    for col in date_columns:
        if col in df_export.columns:
            df_export[col] = df_export[col].apply(lambda x: x.strftime('%d/%m/%Y') if pd.notnull(x) else '')

    # Convert timestamp to Australian format
    if 'event_timestamp_utc' in df_export.columns:
        df_export['event_timestamp_utc'] = df_export['event_timestamp_utc'].apply(
            lambda x: x.strftime('%d/%m/%Y %H:%M:%S') if pd.notnull(x) else ''
        )

    df_export.to_csv(filename, index=False)
    print(f"✅ CSV saved successfully: {filename}")

    # Download file in Colab
    try:
        from google.colab import files
        files.download(filename)
        print("📥 File download initiated in Colab")
    except:
        print("Note: Not running in Colab, file saved locally")

def main():
    """
    Main function optimized for Google Colab
    """
    print("="*80)
    print("BENDIGO AND ADELAIDE BANK PORTFOLIO ANALYTICS DATASET")
    print("Google Colab Version")
    print("="*80)

    # Setup authentication first
    auth_method = setup_colab_authentication()

    if not auth_method:
        print("❌ Authentication failed. Generating dataset and saving CSV only...")
        df = generate_bendigo_adelaide_banking_data()
        save_csv_backup(df)
        return

    # Generate the dataset
    df = generate_bendigo_adelaide_banking_data()

    # Display basic information about the dataset
    print("\n" + "="*80)
    print("DATASET SUMMARY")
    print("="*80)
    print(f"Total Rows: {len(df):,}")
    print(f"Total Columns: {len(df.columns)}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    print("\nColumn Information:")
    print("-" * 50)
    for i, col in enumerate(df.columns, 1):
        dtype = str(df[col].dtype)
        null_count = df[col].isnull().sum()
        print(f"{i:2d}. {col:<35} {dtype:<15} ({null_count:,} nulls)")

    print("\nSample Data (first 5 rows):")
    print("-" * 50)
    print(df.head())

    print("\nFinancial Measures Summary:")
    print("-" * 50)
    financial_cols = ['balance', 'footing', 'current_limit', 'tran_amount']
    print(df[financial_cols].describe())

    # Setup BigQuery and upload
    print("\n" + "="*80)
    print("BIGQUERY UPLOAD")
    print("="*80)

    # Try BigQuery upload
    client, bigquery, NotFound = setup_bigquery_client()

    if client and bigquery and NotFound:
        try:
            # Create table schema
            schema = create_bigquery_table_schema()

            # Create or update table
            create_or_update_bigquery_table(client, FULL_TABLE_ID, schema, bigquery, NotFound)

            # Upload data to BigQuery
            upload_success = upload_to_bigquery(df, client, FULL_TABLE_ID, bigquery, write_disposition="WRITE_TRUNCATE")

            if upload_success:
                print(f"\n✅ SUCCESS! Data uploaded to BigQuery")
                print(f"📊 Table: {FULL_TABLE_ID}")
                print(f"📈 Rows: {len(df):,}")
                print(f"📋 Columns: {len(df.columns)}")

                # Also save CSV for backup
                print("\n💾 Creating CSV backup...")
                save_csv_backup(df)
            else:
                print("\n❌ BigQuery upload failed. Saving CSV instead...")
                save_csv_backup(df)

        except Exception as e:
            print(f"❌ Unexpected error during BigQuery operations: {e}")
            print("Saving CSV backup...")
            save_csv_backup(df)
    else:
        print("\n⚠️  BigQuery setup failed. Saving CSV instead...")
        save_csv_backup(df)

    print("\n" + "="*80)
    print("PROCESSING COMPLETE")
    print("Ready for Analytics and Executive Reporting")
    print("="*80)

if __name__ == "__main__":
    main()


BENDIGO AND ADELAIDE BANK PORTFOLIO ANALYTICS DATASET
Google Colab Version
Setting up Google Colab authentication...
🔑 Attempting user authentication...
✅ User authentication successful!
Generating 150,000 rows of Bendigo and Adelaide Bank portfolio data...
Generating date fields...
Generating customer identification...
Generating customer names...
Generating address information...
Generating relationship manager data...
Generating account information...
Generating product hierarchy...
Generating financial measures...
Generating transaction data...
Generating officer and management data...
Generating KPI measures...
Creating DataFrame...
Applying outliers to recent data...
Dataset generation complete! Generated 150,000 rows with 96 columns.
Date range: 2022-01-29 to 2025-12-31
Outliers applied to 218 recent records (0.1%)

DATASET SUMMARY
Total Rows: 150,000
Total Columns: 96
Memory Usage: 762.2 MB

Column Information:
--------------------------------------------------
 1. evaluate_dat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 File download initiated in Colab

PROCESSING COMPLETE
Ready for Analytics and Executive Reporting


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, date
import string
import warnings

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Configuration parameters
NUM_ROWS = 150000
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2024, 6, 8)

def generate_linkedin_navigator_data():
    """
    Generate comprehensive LinkedIn Sales Navigator dataset
    Returns: pandas DataFrame with 150,000 rows and 36 columns
    """

    print(f"Generating {NUM_ROWS:,} rows of LinkedIn Sales Navigator data...")

    # Australian business context data
    australian_states = ['VIC', 'NSW', 'QLD', 'WA', 'SA', 'TAS', 'NT', 'ACT']

    australian_cities = [
        'Melbourne', 'Sydney', 'Brisbane', 'Perth', 'Adelaide', 'Gold Coast',
        'Newcastle', 'Canberra', 'Sunshine Coast', 'Wollongong', 'Hobart',
        'Geelong', 'Townsville', 'Cairns', 'Darwin', 'Toowoomba', 'Ballarat',
        'Bendigo', 'Albury', 'Launceston', 'Mackay', 'Rockhampton', 'Bunbury'
    ]

    # Business naming components
    business_names = [
        'Superior', 'Golden', 'Progressive', 'Innovation', 'Elite', 'Quality',
        'Modern', 'Heritage', 'Excellence', 'Future', 'Advanced', 'Strategic',
        'Dynamic', 'Professional', 'Premium', 'Global', 'National', 'Regional',
        'Premier', 'Leading', 'Innovative', 'Specialist', 'Expert', 'Prime',
        'Apex', 'Pinnacle', 'Summit', 'Crown', 'Royal', 'Imperial'
    ]

    business_types = [
        'Consulting', 'Manufacturing', 'Services', 'Industries', 'Holdings',
        'Group', 'Enterprises', 'Solutions', 'Capital', 'Ventures',
        'Development', 'Associates', 'Partners', 'Corporation', 'Company'
    ]

    business_structures = ['Pty Ltd', 'Limited', 'Partnership', 'Trust', 'Corporation']

    # Professional names (Australian context)
    first_names = [
        'Michael', 'Sarah', 'David', 'Emma', 'James', 'Lisa', 'Andrew', 'Rachel',
        'Daniel', 'Jennifer', 'Matthew', 'Amanda', 'Christopher', 'Nicole', 'Ryan',
        'Rebecca', 'Mark', 'Jessica', 'Peter', 'Michelle', 'Anthony', 'Stephanie',
        'Paul', 'Laura', 'Steven', 'Catherine', 'John', 'Karen', 'Adam', 'Susan',
        'Kevin', 'Helen', 'Robert', 'Anna', 'Timothy', 'Natalie', 'Jason', 'Claire',
        'Benjamin', 'Samantha', 'Nathan', 'Louise', 'Simon', 'Victoria', 'Luke', 'Tracy'
    ]

    last_names = [
        'Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Miller', 'Davis',
        'Garcia', 'Rodriguez', 'Wilson', 'Martinez', 'Anderson', 'Taylor',
        'Thomas', 'Hernandez', 'Moore', 'Martin', 'Jackson', 'Thompson', 'White',
        'Lopez', 'Lee', 'Gonzalez', 'Harris', 'Clark', 'Lewis', 'Robinson',
        'Walker', 'Perez', 'Hall', 'Young', 'Allen', 'Sanchez', 'Wright',
        'King', 'Scott', 'Green', 'Baker', 'Adams', 'Nelson', 'Hill', 'Ramirez',
        'Campbell', 'Mitchell', 'Roberts', 'Carter', 'Phillips', 'Evans', 'Turner'
    ]

    # Professional titles
    executive_titles = [
        'Chief Executive Officer', 'Chief Financial Officer', 'Chief Technology Officer',
        'Chief Operating Officer', 'Managing Director', 'Executive Director',
        'General Manager', 'Head of Finance', 'Head of Operations', 'Head of Sales',
        'Director of Strategy', 'VP Technology', 'VP Business Development',
        'Finance Director', 'Operations Director', 'Business Development Manager'
    ]

    senior_titles = [
        'Senior Manager', 'Senior Business Analyst', 'Senior Project Manager',
        'Head of Marketing', 'Head of HR', 'Head of Procurement', 'Head of Treasury',
        'Senior Financial Analyst', 'Senior Operations Manager', 'Business Manager',
        'Regional Manager', 'Area Manager', 'Branch Manager', 'Team Leader'
    ]

    mid_titles = [
        'Business Analyst', 'Project Manager', 'Financial Analyst', 'Operations Manager',
        'Marketing Manager', 'Sales Manager', 'Account Manager', 'Product Manager',
        'Business Development Officer', 'Finance Manager', 'Operations Coordinator',
        'Strategy Analyst', 'Investment Analyst', 'Risk Analyst', 'Compliance Officer'
    ]

    # Industries (business banking focus)
    industries = [
        'Management Consulting', 'Manufacturing', 'Professional Services',
        'Investment Management', 'Construction', 'Technology', 'Real Estate',
        'Real Estate Development', 'Venture Capital', 'Mining Services',
        'Engineering Services', 'Financial Services', 'Healthcare Services',
        'Transport & Logistics', 'Retail Trade', 'Wholesale Trade',
        'Agriculture', 'Energy & Utilities', 'Telecommunications', 'Media'
    ]

    # Company sizes
    company_sizes = ['11-50', '51-200', '101-250', '201-500', '251-500', '501-1000', '1001-5000', '5001-10000', '10001+']
    company_size_weights = [0.25, 0.20, 0.15, 0.15, 0.10, 0.08, 0.05, 0.015, 0.005]

    # Technologies
    technologies = [
        'Microsoft 365, Salesforce, QuickBooks', 'SAP, Oracle, Microsoft Teams',
        'AWS, Kubernetes, Python', 'Bloomberg Terminal, Salesforce Financial',
        'Procore, AutoCAD, SAP', 'Microsoft Azure, Power BI, Teams',
        'Property management software, CRM', 'Salesforce, Tableau, Microsoft Office',
        'ERP systems, supply chain management', 'Investment management platforms',
        'Manufacturing execution systems', 'Treasury management systems'
    ]

    # Banking relationships
    australian_banks = [
        'Commonwealth Bank', 'Westpac', 'ANZ', 'NAB', 'Bendigo Bank',
        'Macquarie Bank', 'Suncorp', 'Bank of Queensland', 'ING',
        'Heritage Bank', 'Adelaide Bank', 'Great Southern Bank'
    ]

    # Generate base data structure
    data = {}

    print("Generating names and titles...")

    # Names and titles
    data['First Name'] = np.random.choice(first_names, NUM_ROWS)
    data['Last Name'] = np.random.choice(last_names, NUM_ROWS)

    # Title generation based on seniority levels
    title_categories = np.random.choice(['executive', 'senior', 'mid'], NUM_ROWS, p=[0.15, 0.35, 0.50])
    titles = []
    for category in title_categories:
        if category == 'executive':
            titles.append(random.choice(executive_titles))
        elif category == 'senior':
            titles.append(random.choice(senior_titles))
        else:
            titles.append(random.choice(mid_titles))
    data['Title'] = titles

    print("Generating company information...")

    # Company names
    company_names = []
    for _ in range(NUM_ROWS):
        name1 = random.choice(business_names)
        name2 = random.choice(business_types)
        structure = random.choice(business_structures)
        company_names.append(f"{name1} {name2} {structure}")
    data['Company'] = company_names

    # Contact information (limited availability as per LinkedIn reality)
    emails = []
    phones = []
    for i in range(NUM_ROWS):
        # Only 15% have email, 25% have phone (realistic LinkedIn limitation)
        if random.random() < 0.15:
            first = data['First Name'][i].lower()
            last = data['Last Name'][i].lower()
            domain = random.choice(['gmail.com', 'company.com.au', 'business.com.au'])
            emails.append(f"{first}.{last}@{domain}")
        else:
            emails.append("")

        if random.random() < 0.25:
            area_code = random.choice(['02', '03', '07', '08'])
            number = ''.join([str(random.randint(0, 9)) for _ in range(8)])
            phones.append(f"+61 {area_code} {number[:4]} {number[4:]}")
        else:
            phones.append("")

    data['Email'] = emails
    data['Phone'] = phones

    # LinkedIn URLs
    linkedin_profiles = []
    company_urls = []
    sales_navigator_urls = []

    for i in range(NUM_ROWS):
        first = data['First Name'][i].lower().replace(' ', '-')
        last = data['Last Name'][i].lower().replace(' ', '-')
        company = data['Company'][i].lower().replace(' ', '-').replace('&', 'and')

        linkedin_profiles.append(f"https://www.linkedin.com/in/{first}-{last}-{random.randint(100, 999)}")
        company_urls.append(f"https://www.linkedin.com/company/{company}")
        sales_navigator_urls.append(f"https://www.linkedin.com/sales/lead/ACwAAB{random.choice(string.ascii_uppercase)}{random.randint(100, 999)}")

    data['LinkedIn Profile URL'] = linkedin_profiles
    data['Current Company LinkedIn URL'] = company_urls
    data['Sales Navigator Lead URL'] = sales_navigator_urls

    print("Generating location and company details...")

    # Location data
    # Calculate correct number of remaining cities for weights
    num_remaining_cities = len(australian_cities) - 5
    remaining_weight = 0.22 / num_remaining_cities
    city_weights = [0.25, 0.20, 0.15, 0.10, 0.08] + [remaining_weight] * num_remaining_cities
    data['Location'] = [f"{city}, {random.choice(australian_states)}, Australia"
                       for city in np.random.choice(australian_cities, NUM_ROWS, p=city_weights)]

    # Industry and company size
    data['Industry'] = np.random.choice(industries, NUM_ROWS)
    data['Company Size'] = np.random.choice(company_sizes, NUM_ROWS, p=company_size_weights)

    print("Generating LinkedIn engagement metrics...")

    # LinkedIn engagement metrics
    data['Connection Degree'] = np.random.choice(['1st', '2nd', '3rd'], NUM_ROWS, p=[0.20, 0.50, 0.30])
    data['Shared Connections'] = np.random.randint(0, 50, NUM_ROWS)
    data['Profile Views'] = np.random.randint(5, 100, NUM_ROWS)
    data['Lead Score'] = np.random.randint(60, 100, NUM_ROWS)  # LinkedIn scores typically 60-100
    data['Premium Account'] = np.random.choice(['Yes', 'No'], NUM_ROWS, p=[0.30, 0.70])

    # Professional experience
    data['Years at Company'] = np.round(np.random.exponential(2.5, NUM_ROWS), 1)
    data['Total Experience'] = data['Years at Company'] + np.round(np.random.exponential(8, NUM_ROWS), 1)

    # Education (simplified)
    universities = [
        'University of Melbourne MBA', 'University of Sydney Business', 'UNSW Engineering',
        'Monash University Commerce', 'University of Queensland Business', 'UTS Finance',
        'Griffith University Business', 'Deakin University MBA', 'Curtin University Commerce',
        'Macquarie University Finance', 'Adelaide University Business', 'ANU Economics'
    ]
    data['Education'] = np.random.choice(universities, NUM_ROWS)

    print("Generating business intelligence and activity data...")

    # Recent activity (key for banking opportunities)
    activities = [
        'Posted about cash flow challenges, seeking working capital solutions',
        'Shared article about factory expansion plans, mentioned equipment upgrades',
        'Updated experience with recent acquisition completion, hinting at next targets',
        'Posted about merchant service fees hurting margins, comparing banking providers',
        'Announced new fund launch, seeking banking partners for fund administration',
        'Updated profile highlighting treasury transformation project, seeking technology solutions',
        'Posted about digital transformation budget approval, fintech integration plans',
        'Shared concerns about interest rate impacts, exploring refinancing options',
        'Announced merger discussions with competitor, seeking M&A financing advice',
        'Posted about rapid growth requiring additional banking facilities, scaling challenges',
        'Updated with new property development approval, seeking construction finance',
        'Shared article about supply chain financing challenges, working capital constraints',
        'Posted about expanding into new markets, international banking requirements',
        'Updated experience with ESG investment focus, seeking sustainable finance options',
        'Announced successful exit, reinvesting proceeds, seeking private banking services',
        'Posted about operational efficiency project, seeking technology financing',
        'Shared concerns about client concentration risk, diversification financing needs',
        'Updated profile with acquisition target identified, seeking acquisition finance',
        'Posted about interest rate hedging strategies, derivatives requirements',
        'Announced major export contract win, working capital for production ramp-up needed'
    ]

    # Generate recent activity with realistic timing
    recent_activities = []
    for _ in range(NUM_ROWS):
        days_ago = random.randint(1, 14)
        activity = random.choice(activities)
        recent_activities.append(f"{activity} - {days_ago} days ago")

    data['Last Activity'] = recent_activities

    # Tags and notes (sales-specific)
    tags_options = [
        'Cash Flow Issues;Working Capital Need;High Priority',
        'Manufacturing Growth;Equipment Finance;Expansion',
        'M&A Activity;Acquisition Financing;Strategic Growth',
        'Merchant Services;Banking Review;Cost Optimization',
        'Fund Administration;Banking Partnership;Wealth Management',
        'Treasury Management;Technology Upgrade;Cash Management',
        'Digital Transformation;Fintech Integration;Technology Investment',
        'Refinancing;Interest Rate Risk;Property Finance',
        'Supply Chain Finance;Working Capital;Cash Flow',
        'International Banking;Market Expansion;Foreign Exchange',
        'ESG Finance;Sustainable Investing;Green Loans',
        'Private Banking;Wealth Management;Investment Services'
    ]

    data['Tags'] = np.random.choice(tags_options, NUM_ROWS)

    # Notes (contextual business intelligence)
    notes_templates = [
        'Recent posts indicate operational cash flow challenges, multiple consulting firm payments suggest restructuring costs',
        'CEO actively discussing expansion plans, factory modernization, equipment needs for Q4',
        'M&A specialist, recent acquisition completed, actively seeking next targets in Q3-Q4',
        'CFO publicly discussing merchant service costs, comparing bank offerings, cost reduction focus',
        'New fund launch requires banking services, fund administration, custody solutions',
        'Treasury transformation underway, seeking modern cash management solutions, technology upgrade',
        'CTO approved for digital transformation budget, fintech integration, API banking interest',
        'Finance Director concerned about rate impacts, actively exploring refinancing options',
        'CEO publicly announced merger discussions, actively seeking M&A financing and advisory services',
        'Head of Finance indicating rapid growth, need for additional banking facilities, scaling support'
    ]

    data['Notes'] = np.random.choice(notes_templates, NUM_ROWS)

    print("Generating sales tracking data...")

    # Sales tracking fields
    date_added_list = []
    last_contacted_list = []
    for _ in range(NUM_ROWS):
        # Date added (random over past 6 months)
        days_back = random.randint(30, 180)
        date_added = (datetime.now() - timedelta(days=days_back)).date()
        date_added_list.append(date_added.strftime('%Y-%m-%d'))

        # Last contacted (40% have been contacted)
        if random.random() < 0.4:
            contact_days_back = random.randint(1, days_back)
            last_contact = (datetime.now() - timedelta(days=contact_days_back)).date()
            last_contacted_list.append(last_contact.strftime('%Y-%m-%d'))
        else:
            last_contacted_list.append("")

    data['Date Added'] = date_added_list
    data['Last Contacted'] = last_contacted_list

    # Contact methods and responses
    contact_methods = ['InMail', 'LinkedIn Message', 'Email', 'Phone Call', 'Connection Request']
    response_statuses = [
        'Responded - Interested', 'Responded - Not Interested', 'No Response',
        'Accepted Connection', 'Declined Initial Contact', 'Scheduled Meeting',
        'Follow-up Scheduled', 'Very Interested', 'Very Engaged', 'Opened - No Response'
    ]

    contact_method_list = []
    response_status_list = []

    for i in range(NUM_ROWS):
        if data['Last Contacted'][i]:  # Only if they were contacted
            contact_method_list.append(random.choice(contact_methods))
            response_status_list.append(random.choice(response_statuses))
        else:
            contact_method_list.append("Not Contacted")
            response_status_list.append("")

    data['Contact Method'] = contact_method_list
    data['Response Status'] = response_status_list

    print("Generating financial and business intelligence...")

    # Financial and business intelligence
    # Company revenue (correlated with company size)
    revenues = []
    for size in data['Company Size']:
        if size in ['11-50']:
            revenue = random.uniform(0.5, 5.0)
        elif size in ['51-200', '101-250']:
            revenue = random.uniform(3.0, 20.0)
        elif size in ['201-500', '251-500']:
            revenue = random.uniform(15.0, 80.0)
        elif size in ['501-1000']:
            revenue = random.uniform(50.0, 200.0)
        else:
            revenue = random.uniform(100.0, 1000.0)
        revenues.append(f"${revenue:.1f}M")

    data['Company Revenue'] = revenues

    # Financial health score (60-95)
    data['Financial Health Score'] = np.random.randint(60, 96, NUM_ROWS)

    # Recent funding
    funding_options = ['None', 'Series A funding', 'Series B funding', 'Equipment loan',
                      'Private equity backing', 'Bank facilities', 'Export contract secured']
    funding_weights = [0.6, 0.1, 0.05, 0.1, 0.05, 0.05, 0.05]
    data['Recent Funding'] = np.random.choice(funding_options, NUM_ROWS, p=funding_weights)

    # M&A activity
    ma_options = ['None', 'Acquisition discussions', 'Merger talks active',
                 'Target identified', 'Due diligence active']
    ma_weights = [0.8, 0.08, 0.05, 0.04, 0.03]
    data['Merger Activity'] = np.random.choice(ma_options, NUM_ROWS, p=ma_weights)

    # Competitor banks used (2 banks per company typically)
    competitor_banks = []
    for _ in range(NUM_ROWS):
        primary_bank = random.choice(australian_banks)
        secondary_bank = random.choice([b for b in australian_banks if b != primary_bank])
        competitor_banks.append(f"{primary_bank};{secondary_bank}")

    data['Competitor Banks Used'] = competitor_banks

    # Technology adoption
    tech_levels = ['Basic Technology', 'Modern Tech Stack', 'Legacy Systems', 'Advanced Technology']
    data['Technology Adoption'] = np.random.choice(tech_levels, NUM_ROWS, p=[0.4, 0.3, 0.2, 0.1])

    # Industry trends
    trends = [
        'Professional Services Consolidation', 'Manufacturing Automation Growth',
        'Fintech Integration Accelerating', 'ESG Investment Focus Growing',
        'Supply Chain Digitization', 'Construction Technology Adoption',
        'Real Estate Market Transformation', 'Venture Capital Activity',
        'Export Market Opportunities', 'Interest Rate Risk Management'
    ]
    data['Industry Trends'] = np.random.choice(trends, NUM_ROWS)

    # Key business signals (opportunities)
    business_signals = [
        'Cash flow stress from project delays', 'Large equipment purchases planned',
        'Multiple acquisition discussions', 'Merchant service cost concerns',
        'New fund administration needs', 'Treasury system modernization',
        'Fintech partnership opportunities', 'Refinancing discussions active',
        'Merger financing required', 'Rapid scaling requiring banking support',
        'Major construction project financing', 'Working capital financing needs',
        'International banking needs', 'Green finance requirements',
        'High net worth banking needs', 'Technology upgrade financing'
    ]
    data['Key Business Signals'] = np.random.choice(business_signals, NUM_ROWS)

    # Opportunity types (banking products)
    opportunity_types = [
        'Equipment Finance', 'Business Loan', 'M&A Financing', 'Merchant Services',
        'Wealth Solutions', 'Transaction Banking', 'Digital Banking', 'Property Finance',
        'Supply Chain Finance', 'International Banking', 'Sustainable Finance',
        'Private Banking', 'Construction Finance', 'Export Finance', 'Working Capital',
        'Treasury Solutions', 'Growth Capital', 'Acquisition Finance'
    ]
    data['Opportunity Type'] = np.random.choice(opportunity_types, NUM_ROWS)

    # Risk indicators
    risk_levels = ['Low', 'Medium', 'High']
    risk_descriptions = [
        'Strong financials', 'Cash flow constraints', 'Interest rate exposure',
        'Solid business', 'Supply chain pressures', 'Technology transition',
        'Project-dependent revenue', 'Market concentration', 'Expansion focused',
        'Operational transformation', 'Regulatory compliance', 'Market volatility'
    ]

    risk_indicators = []
    for _ in range(NUM_ROWS):
        level = random.choice(risk_levels)
        description = random.choice(risk_descriptions)
        risk_indicators.append(f"{level} - {description}")

    data['Risk Indicators'] = risk_indicators

    print("Creating DataFrame...")
    df = pd.DataFrame(data)

    print(f"Dataset generation complete! Generated {len(df):,} rows with {len(df.columns)} columns.")

    return df

def main():
    """
    Main function to generate LinkedIn Sales Navigator dataset
    """
    print("="*80)
    print("LINKEDIN SALES NAVIGATOR DATA GENERATOR")
    print("Banking Customer Intelligence Export")
    print("="*80)

    # Generate the dataset
    df = generate_linkedin_navigator_data()

    # Display basic information
    print("\n" + "="*80)
    print("DATASET SUMMARY")
    print("="*80)
    print(f"Total Rows: {len(df):,}")
    print(f"Total Columns: {len(df.columns)}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    print("\nColumn Information:")
    print("-" * 50)
    for i, col in enumerate(df.columns, 1):
        dtype = str(df[col].dtype)
        null_count = df[col].isnull().sum()
        print(f"{i:2d}. {col:<35} {dtype:<15} ({null_count:,} nulls)")

    print("\nSample Data (first 5 rows):")
    print("-" * 50)
    print(df.head())

    print("\nKey Metrics Summary:")
    print("-" * 50)
    print(f"Connection Degrees: {df['Connection Degree'].value_counts().to_dict()}")
    print(f"Company Sizes: {df['Company Size'].value_counts().head().to_dict()}")
    print(f"Industries: {df['Industry'].value_counts().head().to_dict()}")
    print(f"Opportunity Types: {df['Opportunity Type'].value_counts().head().to_dict()}")

    # Save to CSV
    csv_filename = 'linkedin_sales_navigator_banking_150k.csv'
    df.to_csv(csv_filename, index=False)
    print(f"\n📁 Dataset saved as: {csv_filename}")

    # Download in Colab
    try:
        from google.colab import files
        files.download(csv_filename)
        print("📥 File download initiated in Colab")
    except ImportError:
        print("Note: Not running in Colab - file saved locally")

    print("\n" + "="*80)
    print("LINKEDIN SALES NAVIGATOR DATA GENERATION COMPLETE")
    print("Ready for Banking Intelligence Analysis")
    print("="*80)

    return df

# Execute the main function
if __name__ == "__main__":
    df = main()

LINKEDIN SALES NAVIGATOR DATA GENERATOR
Banking Customer Intelligence Export
Generating 150,000 rows of LinkedIn Sales Navigator data...
Generating names and titles...
Generating company information...
Generating location and company details...
Generating LinkedIn engagement metrics...
Generating business intelligence and activity data...
Generating sales tracking data...
Generating financial and business intelligence...
Creating DataFrame...
Dataset generation complete! Generated 150,000 rows with 37 columns.

DATASET SUMMARY
Total Rows: 150,000
Total Columns: 37
Memory Usage: 363.2 MB

Column Information:
--------------------------------------------------
 1. First Name                          object          (0 nulls)
 2. Last Name                           object          (0 nulls)
 3. Title                               object          (0 nulls)
 4. Company                             object          (0 nulls)
 5. Email                               object          (0 nulls)
 6. 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 File download initiated in Colab

LINKEDIN SALES NAVIGATOR DATA GENERATION COMPLETE
Ready for Banking Intelligence Analysis


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, date
import string
import warnings
import os
import json

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Configuration parameters - easily adjustable
NUM_ROWS = 150000
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2024, 6, 8)

# BigQuery configuration
PROJECT_ID = "ehc-ldioneda-1d3210"
DATASET_ID = "larry_demo_datasets"
TABLE_ID = "linkedin_navigator_bb_sales_portfolio_demo"
FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

def setup_colab_authentication():
    """
    Setup authentication in Google Colab environment
    """
    print("Setting up Google Colab authentication...")

    try:
        # Method 1: Try user authentication first
        from google.colab import auth
        print("🔑 Attempting user authentication...")
        auth.authenticate_user()
        print("✅ User authentication successful!")
        return "user"
    except Exception as e:
        print(f"⚠️  User authentication failed: {e}")

    # Method 2: Service account upload
    print("\n📁 Please upload your service account JSON file using the method below:")
    print("1. Click the 'Choose Files' button that appears")
    print("2. Select your service account JSON file")
    print("3. Wait for upload to complete")

    try:
        from google.colab import files
        uploaded = files.upload()

        if uploaded:
            # Get the first uploaded file
            filename = list(uploaded.keys())[0]
            print(f"📄 Uploaded file: {filename}")

            # Set environment variable
            os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = filename
            print("✅ Service account credentials configured!")
            return "service_account"
        else:
            print("❌ No file uploaded")
            return None

    except Exception as e:
        print(f"❌ File upload failed: {e}")
        return None

def setup_bigquery_client():
    """
    Setup BigQuery client for Google Colab
    """
    try:
        from google.cloud import bigquery
        from google.cloud.exceptions import NotFound
        from google.auth.exceptions import DefaultCredentialsError

        print(f"Setting up BigQuery client for project: {PROJECT_ID}")

        # Try to create client
        client = bigquery.Client(project=PROJECT_ID)

        # Test the connection
        try:
            dataset = client.get_dataset(DATASET_ID)
            print(f"✅ Successfully connected to dataset: {DATASET_ID}")
            return client, bigquery, NotFound
        except Exception as e:
            print(f"⚠️  Dataset access test failed: {e}")
            print("Note: This might be normal if the dataset exists but has restricted access")
            return client, bigquery, NotFound

    except ImportError:
        print("❌ google-cloud-bigquery not installed. Installing now...")
        !pip install google-cloud-bigquery

        from google.cloud import bigquery
        from google.cloud.exceptions import NotFound
        client = bigquery.Client(project=PROJECT_ID)
        return client, bigquery, NotFound

    except Exception as e:
        print(f"❌ Error setting up BigQuery client: {e}")
        print("\nTrying alternative authentication...")
        return None, None, None

def create_bigquery_table_schema():
    """
    Define BigQuery table schema for LinkedIn Sales Navigator data
    """
    # Import here in case it's not available at module level
    from google.cloud import bigquery

    schema = [
        bigquery.SchemaField("first_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("last_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("company", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("email", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("phone", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("linkedin_profile_url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("location", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("industry", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("company_size", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("connection_degree", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("shared_connections", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("last_activity", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("profile_views", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("lead_score", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("premium_account", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("years_at_company", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("total_experience", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("education", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("current_company_linkedin_url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("sales_navigator_lead_url", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("tags", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("notes", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("date_added", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("last_contacted", "DATE", mode="NULLABLE"),
        bigquery.SchemaField("contact_method", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("response_status", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("company_revenue", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("financial_health_score", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("recent_funding", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("merger_activity", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("competitor_banks_used", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("technology_adoption", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("industry_trends", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("key_business_signals", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("opportunity_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("risk_indicators", "STRING", mode="NULLABLE"),
    ]
    return schema

def create_or_update_bigquery_table(client, table_id, schema, bigquery, NotFound):
    """
    Create BigQuery table if it doesn't exist, or update schema if it does
    """
    try:
        # Check if table exists
        table = client.get_table(table_id)
        print(f"Table {table_id} already exists. Checking schema...")

        # Check if schema needs updating
        current_schema = table.schema
        if len(current_schema) != len(schema):
            print("Schema differs. Updating table schema...")
            table.schema = schema
            table = client.update_table(table, ["schema"])
            print("Table schema updated successfully.")
        else:
            print("Table schema is up to date.")

    except NotFound:
        # Table doesn't exist, create it
        print(f"Creating table {table_id}...")
        table = bigquery.Table(table_id, schema=schema)
        table = client.create_table(table)
        print(f"Table {table_id} created successfully.")

    return table

def upload_to_bigquery(df, client, table_id, bigquery, write_disposition="WRITE_TRUNCATE"):
    """
    Upload DataFrame to BigQuery table
    """
    # Configure the job
    job_config = bigquery.LoadJobConfig(
        write_disposition=write_disposition,
        autodetect=False,  # Use our explicit schema
        schema=create_bigquery_table_schema()
    )

    print(f"Starting upload to {table_id}...")
    print(f"Upload mode: {write_disposition}")
    print(f"Rows to upload: {len(df):,}")

    # Start the job
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

    # Wait for the job to complete
    try:
        job.result()  # Wait for job to complete
        print(f"✅ Successfully uploaded {len(df):,} rows to {table_id}")

        # Get table info
        table = client.get_table(table_id)
        print(f"📊 Table now contains {table.num_rows:,} total rows")
        return True

    except Exception as e:
        print(f"❌ Error uploading to BigQuery: {e}")
        if hasattr(job, 'errors') and job.errors:
            for error in job.errors:
                print(f"Job error: {error}")
        return False

def generate_linkedin_navigator_data():
    """
    Generate comprehensive LinkedIn Sales Navigator dataset for banking customers
    Returns: pandas DataFrame with 150,000 rows and 37 columns
    """

    print(f"Generating {NUM_ROWS:,} rows of LinkedIn Sales Navigator data...")

    # Australian business context data
    australian_states = ['VIC', 'NSW', 'QLD', 'WA', 'SA', 'TAS', 'NT', 'ACT']

    australian_cities = [
        'Melbourne', 'Sydney', 'Brisbane', 'Perth', 'Adelaide', 'Gold Coast',
        'Newcastle', 'Canberra', 'Sunshine Coast', 'Wollongong', 'Hobart',
        'Geelong', 'Townsville', 'Cairns', 'Darwin', 'Toowoomba', 'Ballarat',
        'Bendigo', 'Albury', 'Launceston', 'Mackay', 'Rockhampton', 'Bunbury'
    ]

    # Business naming components
    business_names = [
        'Superior', 'Golden', 'Progressive', 'Innovation', 'Elite', 'Quality',
        'Modern', 'Heritage', 'Excellence', 'Future', 'Advanced', 'Strategic',
        'Dynamic', 'Professional', 'Premium', 'Global', 'National', 'Regional',
        'Premier', 'Leading', 'Innovative', 'Specialist', 'Expert', 'Prime',
        'Apex', 'Pinnacle', 'Summit', 'Crown', 'Royal', 'Imperial'
    ]

    business_types = [
        'Consulting', 'Manufacturing', 'Services', 'Industries', 'Holdings',
        'Group', 'Enterprises', 'Solutions', 'Capital', 'Ventures',
        'Development', 'Associates', 'Partners', 'Corporation', 'Company'
    ]

    business_structures = ['Pty Ltd', 'Limited', 'Partnership', 'Trust', 'Corporation']

    # Professional names (Australian context)
    first_names = [
        'Michael', 'Sarah', 'David', 'Emma', 'James', 'Lisa', 'Andrew', 'Rachel',
        'Daniel', 'Jennifer', 'Matthew', 'Amanda', 'Christopher', 'Nicole', 'Ryan',
        'Rebecca', 'Mark', 'Jessica', 'Peter', 'Michelle', 'Anthony', 'Stephanie',
        'Paul', 'Laura', 'Steven', 'Catherine', 'John', 'Karen', 'Adam', 'Susan',
        'Kevin', 'Helen', 'Robert', 'Anna', 'Timothy', 'Natalie', 'Jason', 'Claire',
        'Benjamin', 'Samantha', 'Nathan', 'Louise', 'Simon', 'Victoria', 'Luke', 'Tracy'
    ]

    last_names = [
        'Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Miller', 'Davis',
        'Garcia', 'Rodriguez', 'Wilson', 'Martinez', 'Anderson', 'Taylor',
        'Thomas', 'Hernandez', 'Moore', 'Martin', 'Jackson', 'Thompson', 'White',
        'Lopez', 'Lee', 'Gonzalez', 'Harris', 'Clark', 'Lewis', 'Robinson',
        'Walker', 'Perez', 'Hall', 'Young', 'Allen', 'Sanchez', 'Wright',
        'King', 'Scott', 'Green', 'Baker', 'Adams', 'Nelson', 'Hill', 'Ramirez',
        'Campbell', 'Mitchell', 'Roberts', 'Carter', 'Phillips', 'Evans', 'Turner'
    ]

    # Professional titles
    executive_titles = [
        'Chief Executive Officer', 'Chief Financial Officer', 'Chief Technology Officer',
        'Chief Operating Officer', 'Managing Director', 'Executive Director',
        'General Manager', 'Head of Finance', 'Head of Operations', 'Head of Sales',
        'Director of Strategy', 'VP Technology', 'VP Business Development',
        'Finance Director', 'Operations Director', 'Business Development Manager'
    ]

    senior_titles = [
        'Senior Manager', 'Senior Business Analyst', 'Senior Project Manager',
        'Head of Marketing', 'Head of HR', 'Head of Procurement', 'Head of Treasury',
        'Senior Financial Analyst', 'Senior Operations Manager', 'Business Manager',
        'Regional Manager', 'Area Manager', 'Branch Manager', 'Team Leader'
    ]

    mid_titles = [
        'Business Analyst', 'Project Manager', 'Financial Analyst', 'Operations Manager',
        'Marketing Manager', 'Sales Manager', 'Account Manager', 'Product Manager',
        'Business Development Officer', 'Finance Manager', 'Operations Coordinator',
        'Strategy Analyst', 'Investment Analyst', 'Risk Analyst', 'Compliance Officer'
    ]

    # Industries (business banking focus)
    industries = [
        'Management Consulting', 'Manufacturing', 'Professional Services',
        'Investment Management', 'Construction', 'Technology', 'Real Estate',
        'Real Estate Development', 'Venture Capital', 'Mining Services',
        'Engineering Services', 'Financial Services', 'Healthcare Services',
        'Transport & Logistics', 'Retail Trade', 'Wholesale Trade',
        'Agriculture', 'Energy & Utilities', 'Telecommunications', 'Media'
    ]

    # Company sizes
    company_sizes = ['11-50', '51-200', '101-250', '201-500', '251-500', '501-1000', '1001-5000', '5001-10000', '10001+']
    company_size_weights = [0.25, 0.20, 0.15, 0.15, 0.10, 0.08, 0.05, 0.015, 0.005]

    # Australian banks
    australian_banks = [
        'Commonwealth Bank', 'Westpac', 'ANZ', 'NAB', 'Bendigo Bank',
        'Macquarie Bank', 'Suncorp', 'Bank of Queensland', 'ING',
        'Heritage Bank', 'Adelaide Bank', 'Great Southern Bank'
    ]

    # Generate base data structure
    data = {}

    print("Generating names and titles...")

    # Names and titles
    data['first_name'] = np.random.choice(first_names, NUM_ROWS)
    data['last_name'] = np.random.choice(last_names, NUM_ROWS)

    # Title generation based on seniority levels
    title_categories = np.random.choice(['executive', 'senior', 'mid'], NUM_ROWS, p=[0.15, 0.35, 0.50])
    titles = []
    for category in title_categories:
        if category == 'executive':
            titles.append(random.choice(executive_titles))
        elif category == 'senior':
            titles.append(random.choice(senior_titles))
        else:
            titles.append(random.choice(mid_titles))
    data['title'] = titles

    print("Generating company information...")

    # Company names
    company_names = []
    for _ in range(NUM_ROWS):
        name1 = random.choice(business_names)
        name2 = random.choice(business_types)
        structure = random.choice(business_structures)
        company_names.append(f"{name1} {name2} {structure}")
    data['company'] = company_names

    # Contact information (limited availability as per LinkedIn reality)
    emails = []
    phones = []
    for i in range(NUM_ROWS):
        # Only 15% have email, 25% have phone (realistic LinkedIn limitation)
        if random.random() < 0.15:
            first = data['first_name'][i].lower()
            last = data['last_name'][i].lower()
            domain = random.choice(['gmail.com', 'company.com.au', 'business.com.au'])
            emails.append(f"{first}.{last}@{domain}")
        else:
            emails.append("")

        if random.random() < 0.25:
            area_code = random.choice(['02', '03', '07', '08'])
            number = ''.join([str(random.randint(0, 9)) for _ in range(8)])
            phones.append(f"+61 {area_code} {number[:4]} {number[4:]}")
        else:
            phones.append("")

    data['email'] = emails
    data['phone'] = phones

    # LinkedIn URLs
    linkedin_profiles = []
    company_urls = []
    sales_navigator_urls = []

    for i in range(NUM_ROWS):
        first = data['first_name'][i].lower().replace(' ', '-')
        last = data['last_name'][i].lower().replace(' ', '-')
        company = data['company'][i].lower().replace(' ', '-').replace('&', 'and')

        linkedin_profiles.append(f"https://www.linkedin.com/in/{first}-{last}-{random.randint(100, 999)}")
        company_urls.append(f"https://www.linkedin.com/company/{company}")
        sales_navigator_urls.append(f"https://www.linkedin.com/sales/lead/ACwAAB{random.choice(string.ascii_uppercase)}{random.randint(100, 999)}")

    data['linkedin_profile_url'] = linkedin_profiles
    data['current_company_linkedin_url'] = company_urls
    data['sales_navigator_lead_url'] = sales_navigator_urls

    print("Generating location and company details...")

    # Location data
    num_remaining_cities = len(australian_cities) - 5
    remaining_weight = 0.22 / num_remaining_cities
    city_weights = [0.25, 0.20, 0.15, 0.10, 0.08] + [remaining_weight] * num_remaining_cities
    data['location'] = [f"{city}, {random.choice(australian_states)}, Australia"
                       for city in np.random.choice(australian_cities, NUM_ROWS, p=city_weights)]

    # Industry and company size
    data['industry'] = np.random.choice(industries, NUM_ROWS)
    data['company_size'] = np.random.choice(company_sizes, NUM_ROWS, p=company_size_weights)

    print("Generating LinkedIn engagement metrics...")

    # LinkedIn engagement metrics
    data['connection_degree'] = np.random.choice(['1st', '2nd', '3rd'], NUM_ROWS, p=[0.20, 0.50, 0.30])
    data['shared_connections'] = np.random.randint(0, 50, NUM_ROWS)
    data['profile_views'] = np.random.randint(5, 100, NUM_ROWS)
    data['lead_score'] = np.random.randint(60, 100, NUM_ROWS)  # LinkedIn scores typically 60-100
    data['premium_account'] = np.random.choice(['Yes', 'No'], NUM_ROWS, p=[0.30, 0.70])

    # Professional experience
    data['years_at_company'] = np.round(np.random.exponential(2.5, NUM_ROWS), 1)
    data['total_experience'] = data['years_at_company'] + np.round(np.random.exponential(8, NUM_ROWS), 1)

    # Education (simplified)
    universities = [
        'University of Melbourne MBA', 'University of Sydney Business', 'UNSW Engineering',
        'Monash University Commerce', 'University of Queensland Business', 'UTS Finance',
        'Griffith University Business', 'Deakin University MBA', 'Curtin University Commerce',
        'Macquarie University Finance', 'Adelaide University Business', 'ANU Economics'
    ]
    data['education'] = np.random.choice(universities, NUM_ROWS)

    print("Generating business intelligence and activity data...")

    # Recent activity (key for banking opportunities)
    activities = [
        'Posted about cash flow challenges, seeking working capital solutions',
        'Shared article about factory expansion plans, mentioned equipment upgrades',
        'Updated experience with recent acquisition completion, hinting at next targets',
        'Posted about merchant service fees hurting margins, comparing banking providers',
        'Announced new fund launch, seeking banking partners for fund administration',
        'Updated profile highlighting treasury transformation project, seeking technology solutions',
        'Posted about digital transformation budget approval, fintech integration plans',
        'Shared concerns about interest rate impacts, exploring refinancing options',
        'Announced merger discussions with competitor, seeking M&A financing advice',
        'Posted about rapid growth requiring additional banking facilities, scaling challenges',
        'Updated with new property development approval, seeking construction finance',
        'Shared article about supply chain financing challenges, working capital constraints',
        'Posted about expanding into new markets, international banking requirements',
        'Updated experience with ESG investment focus, seeking sustainable finance options',
        'Announced successful exit, reinvesting proceeds, seeking private banking services',
        'Posted about operational efficiency project, seeking technology financing',
        'Shared concerns about client concentration risk, diversification financing needs',
        'Updated profile with acquisition target identified, seeking acquisition finance',
        'Posted about interest rate hedging strategies, derivatives requirements',
        'Announced major export contract win, working capital for production ramp-up needed'
    ]

    # Generate recent activity with realistic timing
    recent_activities = []
    for _ in range(NUM_ROWS):
        days_ago = random.randint(1, 14)
        activity = random.choice(activities)
        recent_activities.append(f"{activity} - {days_ago} days ago")

    data['last_activity'] = recent_activities

    # Tags and notes (sales-specific)
    tags_options = [
        'Cash Flow Issues;Working Capital Need;High Priority',
        'Manufacturing Growth;Equipment Finance;Expansion',
        'M&A Activity;Acquisition Financing;Strategic Growth',
        'Merchant Services;Banking Review;Cost Optimization',
        'Fund Administration;Banking Partnership;Wealth Management',
        'Treasury Management;Technology Upgrade;Cash Management',
        'Digital Transformation;Fintech Integration;Technology Investment',
        'Refinancing;Interest Rate Risk;Property Finance',
        'Supply Chain Finance;Working Capital;Cash Flow',
        'International Banking;Market Expansion;Foreign Exchange',
        'ESG Finance;Sustainable Investing;Green Loans',
        'Private Banking;Wealth Management;Investment Services'
    ]

    data['tags'] = np.random.choice(tags_options, NUM_ROWS)

    # Notes (contextual business intelligence)
    notes_templates = [
        'Recent posts indicate operational cash flow challenges, multiple consulting firm payments suggest restructuring costs',
        'CEO actively discussing expansion plans, factory modernization, equipment needs for Q4',
        'M&A specialist, recent acquisition completed, actively seeking next targets in Q3-Q4',
        'CFO publicly discussing merchant service costs, comparing bank offerings, cost reduction focus',
        'New fund launch requires banking services, fund administration, custody solutions',
        'Treasury transformation underway, seeking modern cash management solutions, technology upgrade',
        'CTO approved for digital transformation budget, fintech integration, API banking interest',
        'Finance Director concerned about rate impacts, actively exploring refinancing options',
        'CEO publicly announced merger discussions, actively seeking M&A financing and advisory services',
        'Head of Finance indicating rapid growth, need for additional banking facilities, scaling support'
    ]

    data['notes'] = np.random.choice(notes_templates, NUM_ROWS)

    print("Generating sales tracking data...")

    # Sales tracking fields
    date_added_list = []
    last_contacted_list = []
    for _ in range(NUM_ROWS):
        # Date added (random over past 6 months)
        days_back = random.randint(30, 180)
        date_added = (datetime.now() - timedelta(days=days_back)).date()
        date_added_list.append(date_added)

        # Last contacted (40% have been contacted)
        if random.random() < 0.4:
            contact_days_back = random.randint(1, days_back)
            last_contact = (datetime.now() - timedelta(days=contact_days_back)).date()
            last_contacted_list.append(last_contact)
        else:
            last_contacted_list.append(None)

    data['date_added'] = date_added_list
    data['last_contacted'] = last_contacted_list

    # Contact methods and responses
    contact_methods = ['InMail', 'LinkedIn Message', 'Email', 'Phone Call', 'Connection Request']
    response_statuses = [
        'Responded - Interested', 'Responded - Not Interested', 'No Response',
        'Accepted Connection', 'Declined Initial Contact', 'Scheduled Meeting',
        'Follow-up Scheduled', 'Very Interested', 'Very Engaged', 'Opened - No Response'
    ]

    contact_method_list = []
    response_status_list = []

    for i in range(NUM_ROWS):
        if data['last_contacted'][i]:  # Only if they were contacted
            contact_method_list.append(random.choice(contact_methods))
            response_status_list.append(random.choice(response_statuses))
        else:
            contact_method_list.append("Not Contacted")
            response_status_list.append("")

    data['contact_method'] = contact_method_list
    data['response_status'] = response_status_list

    print("Generating financial and business intelligence...")

    # Financial and business intelligence
    # Company revenue (correlated with company size)
    revenues = []
    for size in data['company_size']:
        if size in ['11-50']:
            revenue = random.uniform(0.5, 5.0)
        elif size in ['51-200', '101-250']:
            revenue = random.uniform(3.0, 20.0)
        elif size in ['201-500', '251-500']:
            revenue = random.uniform(15.0, 80.0)
        elif size in ['501-1000']:
            revenue = random.uniform(50.0, 200.0)
        else:
            revenue = random.uniform(100.0, 1000.0)
        revenues.append(f"${revenue:.1f}M")

    data['company_revenue'] = revenues

    # Financial health score (60-95)
    data['financial_health_score'] = np.random.randint(60, 96, NUM_ROWS)

    # Recent funding
    funding_options = ['None', 'Series A funding', 'Series B funding', 'Equipment loan',
                      'Private equity backing', 'Bank facilities', 'Export contract secured']
    funding_weights = [0.6, 0.1, 0.05, 0.1, 0.05, 0.05, 0.05]
    data['recent_funding'] = np.random.choice(funding_options, NUM_ROWS, p=funding_weights)

    # M&A activity
    ma_options = ['None', 'Acquisition discussions', 'Merger talks active',
                 'Target identified', 'Due diligence active']
    ma_weights = [0.8, 0.08, 0.05, 0.04, 0.03]
    data['merger_activity'] = np.random.choice(ma_options, NUM_ROWS, p=ma_weights)

    # Competitor banks used (2 banks per company typically)
    competitor_banks = []
    for _ in range(NUM_ROWS):
        primary_bank = random.choice(australian_banks)
        secondary_bank = random.choice([b for b in australian_banks if b != primary_bank])
        competitor_banks.append(f"{primary_bank};{secondary_bank}")

    data['competitor_banks_used'] = competitor_banks

    # Technology adoption
    tech_levels = ['Basic Technology', 'Modern Tech Stack', 'Legacy Systems', 'Advanced Technology']
    data['technology_adoption'] = np.random.choice(tech_levels, NUM_ROWS, p=[0.4, 0.3, 0.2, 0.1])

    # Industry trends
    trends = [
        'Professional Services Consolidation', 'Manufacturing Automation Growth',
        'Fintech Integration Accelerating', 'ESG Investment Focus Growing',
        'Supply Chain Digitization', 'Construction Technology Adoption',
        'Real Estate Market Transformation', 'Venture Capital Activity',
        'Export Market Opportunities', 'Interest Rate Risk Management'
    ]
    data['industry_trends'] = np.random.choice(trends, NUM_ROWS)

    # Key business signals (opportunities)
    business_signals = [
        'Cash flow stress from project delays', 'Large equipment purchases planned',
        'Multiple acquisition discussions', 'Merchant service cost concerns',
        'New fund administration needs', 'Treasury system modernization',
        'Fintech partnership opportunities', 'Refinancing discussions active',
        'Merger financing required', 'Rapid scaling requiring banking support',
        'Major construction project financing', 'Working capital financing needs',
        'International banking needs', 'Green finance requirements',
        'High net worth banking needs', 'Technology upgrade financing'
    ]
    data['key_business_signals'] = np.random.choice(business_signals, NUM_ROWS)

    # Opportunity types (banking products)
    opportunity_types = [
        'Equipment Finance', 'Business Loan', 'M&A Financing', 'Merchant Services',
        'Wealth Solutions', 'Transaction Banking', 'Digital Banking', 'Property Finance',
        'Supply Chain Finance', 'International Banking', 'Sustainable Finance',
        'Private Banking', 'Construction Finance', 'Export Finance', 'Working Capital',
        'Treasury Solutions', 'Growth Capital', 'Acquisition Finance'
    ]
    data['opportunity_type'] = np.random.choice(opportunity_types, NUM_ROWS)

    # Risk indicators
    risk_levels = ['Low', 'Medium', 'High']
    risk_descriptions = [
        'Strong financials', 'Cash flow constraints', 'Interest rate exposure',
        'Solid business', 'Supply chain pressures', 'Technology transition',
        'Project-dependent revenue', 'Market concentration', 'Expansion focused',
        'Operational transformation', 'Regulatory compliance', 'Market volatility'
    ]

    risk_indicators = []
    for _ in range(NUM_ROWS):
        level = random.choice(risk_levels)
        description = random.choice(risk_descriptions)
        risk_indicators.append(f"{level} - {description}")

    data['risk_indicators'] = risk_indicators

    print("Creating DataFrame...")
    df = pd.DataFrame(data)

    print(f"Dataset generation complete! Generated {len(df):,} rows with {len(df.columns)} columns.")

    return df

def save_csv_backup(df, filename='linkedin_sales_navigator_banking_150k.csv'):
    """
    Save DataFrame as CSV
    """
    print(f"\n📁 Saving CSV backup: {filename}")

    df_export = df.copy()

    # Convert date columns to string format for CSV export
    date_columns = ['date_added', 'last_contacted']

    for col in date_columns:
        if col in df_export.columns:
            df_export[col] = df_export[col].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else '')

    df_export.to_csv(filename, index=False)
    print(f"✅ CSV saved successfully: {filename}")

    # Download file in Colab
    try:
        from google.colab import files
        files.download(filename)
        print("📥 File download initiated in Colab")
    except:
        print("Note: Not running in Colab, file saved locally")

def main():
    """
    Main function optimized for Google Colab
    """
    print("="*80)
    print("LINKEDIN SALES NAVIGATOR DATA GENERATOR")
    print("Banking Customer Intelligence Export")
    print("Google Colab BigQuery Version")
    print("="*80)

    # Setup authentication first
    auth_method = setup_colab_authentication()

    if not auth_method:
        print("❌ Authentication failed. Generating dataset and saving CSV only...")
        df = generate_linkedin_navigator_data()
        save_csv_backup(df)
        return

    # Generate the dataset
    df = generate_linkedin_navigator_data()

    # Display basic information about the dataset
    print("\n" + "="*80)
    print("DATASET SUMMARY")
    print("="*80)
    print(f"Total Rows: {len(df):,}")
    print(f"Total Columns: {len(df.columns)}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    print("\nColumn Information:")
    print("-" * 50)
    for i, col in enumerate(df.columns, 1):
        dtype = str(df[col].dtype)
        null_count = df[col].isnull().sum()
        print(f"{i:2d}. {col:<35} {dtype:<15} ({null_count:,} nulls)")

    print("\nSample Data (first 5 rows):")
    print("-" * 50)
    print(df.head())

    print("\nKey Metrics Summary:")
    print("-" * 50)
    print(f"Connection Degrees: {df['connection_degree'].value_counts().to_dict()}")
    print(f"Company Sizes: {df['company_size'].value_counts().head().to_dict()}")
    print(f"Industries: {df['industry'].value_counts().head().to_dict()}")
    print(f"Opportunity Types: {df['opportunity_type'].value_counts().head().to_dict()}")

    # Setup BigQuery and upload
    print("\n" + "="*80)
    print("BIGQUERY UPLOAD")
    print("="*80)

    # Try BigQuery upload
    client, bigquery, NotFound = setup_bigquery_client()

    if client and bigquery and NotFound:
        try:
            # Create table schema
            schema = create_bigquery_table_schema()

            # Create or update table
            create_or_update_bigquery_table(client, FULL_TABLE_ID, schema, bigquery, NotFound)

            # Upload data to BigQuery
            upload_success = upload_to_bigquery(df, client, FULL_TABLE_ID, bigquery, write_disposition="WRITE_TRUNCATE")

            if upload_success:
                print(f"\n✅ SUCCESS! Data uploaded to BigQuery")
                print(f"📊 Table: {FULL_TABLE_ID}")
                print(f"📈 Rows: {len(df):,}")
                print(f"📋 Columns: {len(df.columns)}")

                # Also save CSV for backup
                print("\n💾 Creating CSV backup...")
                save_csv_backup(df)
            else:
                print("\n❌ BigQuery upload failed. Saving CSV instead...")
                save_csv_backup(df)

        except Exception as e:
            print(f"❌ Unexpected error during BigQuery operations: {e}")
            print("Saving CSV backup...")
            save_csv_backup(df)
    else:
        print("\n⚠️  BigQuery setup failed. Saving CSV instead...")
        save_csv_backup(df)

    print("\n" + "="*80)
    print("PROCESSING COMPLETE")
    print("Ready for Banking Intelligence Analysis")
    print("="*80)

if __name__ == "__main__":
    main()

LINKEDIN SALES NAVIGATOR DATA GENERATOR
Banking Customer Intelligence Export
Google Colab BigQuery Version
Setting up Google Colab authentication...
🔑 Attempting user authentication...
✅ User authentication successful!
Generating 150,000 rows of LinkedIn Sales Navigator data...
Generating names and titles...
Generating company information...
Generating location and company details...
Generating LinkedIn engagement metrics...
Generating business intelligence and activity data...
Generating sales tracking data...
Generating financial and business intelligence...
Creating DataFrame...
Dataset generation complete! Generated 150,000 rows with 37 columns.

DATASET SUMMARY
Total Rows: 150,000
Total Columns: 37
Memory Usage: 354.9 MB

Column Information:
--------------------------------------------------
 1. first_name                          object          (0 nulls)
 2. last_name                           object          (0 nulls)
 3. title                               object          (0 n

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 File download initiated in Colab

PROCESSING COMPLETE
Ready for Banking Intelligence Analysis
